### Image classification

**Rationale**: Predict the type product based on images of merchandise


In [1]:
// ML.NET Nuget packages 
#r "nuget:Microsoft.ML,1.4.0"
#r "nuget:Microsoft.ML.Vision"
#r "nuget:Microsoft.ML.ImageAnalytics,1.4.0"
#r "nuget:SciSharp.TensorFlow.Redist,1.14.0"
    
//CSV Helper
#r "nuget:CsvHelper"
    
//Install XPlot package
#r "nuget:XPlot.Plotly,2.0.0"

using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms;
using Microsoft.ML.Vision;
using XPlot.Plotly;
using CsvHelper;
using System.IO;

Installing package Microsoft.ML, version 1.4.0.....................done!

Successfully added reference to package Microsoft.ML, version 1.4.0

Installing package Microsoft.ML.Vision.........done!

Successfully added reference to package Microsoft.ML.Vision, version 1.4.0

Installing package Microsoft.ML.ImageAnalytics, version 1.4.0.......done!

...

Successfully added reference to package SciSharp.TensorFlow.Redist, version 1.14.0

Installing package CsvHelper.........done!

Successfully added reference to package CsvHelper, version 12.2.1

Installing package XPlot.Plotly, version 2.0.0........done!

Successfully added reference to package XPlot.Plotly, version 2.0.0

In [2]:
public class ImageData
{
    public string ImagePath { get; set; }

    public string Label { get; set; }
}

In [3]:
public class Article
{
    public string Id { get; set; }
    public string Gender { get; set; }
    public string MasterCategory { get; set; }
    public string SubCategory { get; set; }
    public string ArticleType { get; set; }
    public string BaseColor { get; set; }
    public string Season { get; set; }
    public string Year { get; set; }
    public string Usage { get; set; }
    public string ProductDisplayName { get; set; }
}

In [22]:
//Please note to change the ImagePath here to your local path of the repo. Tensorflow needs an absolute path instead of a relative path
var imagePath = @"C:\Git\mlnet-jupyter\src\Datasets\Computer Vision\fashion-product-images-small\images";
var csvPath = @".\Datasets\Computer Vision\fashion-product-images-small\testtrain.csv";

In [23]:
public IEnumerable<ImageData> GetImages(string path)
{
    using (var reader = new StreamReader(path))
    using (var csv = new CsvReader(reader))
    {
        return csv.GetRecords<Article>()
            .Select(x => new ImageData
            {
                Label = x.ArticleType,
                ImagePath = Path.Combine(imagePath, x.Id) + ".jpg"
            })
            .Where(y => File.Exists(y.ImagePath))
            .ToList();
    }
}

In [24]:
IEnumerable<ImageData> images = GetImages(csvPath);

In [25]:
var mlContext = new MLContext(seed: 1);

var dataView = mlContext.Data.LoadFromEnumerable(images);
dataView = mlContext.Data.ShuffleRows(dataView);

#### Load images in to memory

In [26]:
IDataView shuffledFullImagesDataset = mlContext.Transforms.Conversion.MapValueToKey(outputColumnName: "LabelAsKey",
                                        inputColumnName: "Label",
                                        keyOrdinality: ValueToKeyMappingEstimator.KeyOrdinality.ByValue)
                                        .Append(mlContext.Transforms.LoadRawImageBytes(outputColumnName: "Image",
                                                                    imageFolder: imagePath,
                                                                    inputColumnName: "ImagePath"))
                                        .Fit(dataView)
                                        .Transform(dataView);

#### Data exploration

#### Plotting

In [27]:
var categories = shuffledFullImagesDataset.GetColumn<string>("Label");

var categoriesHistogram = Chart.Plot(new Graph.Histogram 
    {
        x = categories,  
    });

categoriesHistogram.WithLayout(new Layout.Layout { title="Types of images" });
display(categoriesHistogram);

#### Split our data

In [28]:
var trainTestSplit = mlContext.Data.TrainTestSplit(shuffledFullImagesDataset);

var testSet = trainTestSplit.TestSet;
var trainSet = trainTestSplit.TrainSet;

#### Data Transformation

In [29]:
var options = new ImageClassificationTrainer.Options()
{
    FeatureColumnName = "Image",
    LabelColumnName = "LabelAsKey",
    Arch = ImageClassificationTrainer.Architecture.ResnetV250,
    Epoch = 30,
    BatchSize = 10,
    LearningRate = 0.01f,
    MetricsCallback = (metrics) => Console.WriteLine(metrics),
    ValidationSet = testSet
};

var trainingPipeline = mlContext.MulticlassClassification.Trainers.ImageClassification(options)
        .Append(mlContext.Transforms.Conversion.MapKeyToValue(
            outputColumnName: "PredictedLabel",
            inputColumnName: "PredictedLabel"));

##### Train the model

In [30]:
ITransformer model = trainingPipeline.Fit(trainSet);

Saver not created because there are no variables in the graph to restore
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:   1
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:   2
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:   3
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:   4
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:   5
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:   6
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:   7
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:   8
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:   9
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:  10
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:  11
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:  12
Phase: Bottleneck Computation, Dataset 

Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 111
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 112
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 113
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 114
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 115
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 116
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 117
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 118
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 119
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 120
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 121
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 122
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 123
Phase: Bottleneck Computation, Dataset

Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 222
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 223
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 224
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 225
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 226
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 227
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 228
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 229
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 230
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 231
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 232
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 233
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 234
Phase: Bottleneck Computation, Dataset

Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 333
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 334
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 335
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 336
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 337
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 338
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 339
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 340
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 341
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 342
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 343
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 344
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 345
Phase: Bottleneck Computation, Dataset

Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 444
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 445
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 446
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 447
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 448
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 449
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 450
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 451
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 452
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 453
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 454
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 455
Phase: Bottleneck Computation, Dataset used:      Train, Image Index: 456
Phase: Bottleneck Computation, Dataset

...

Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 305
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 306
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 307
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 308
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 309
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 310
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 311
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 312
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 313
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 314
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 315
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 316
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 317
Phase: Bottleneck Computation, Dataset

Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 416
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 417
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 418
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 419
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 420
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 421
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 422
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 423
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 424
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 425
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 426
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 427
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 428
Phase: Bottleneck Computation, Dataset

Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 527
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 528
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 529
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 530
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 531
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 532
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 533
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 534
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 535
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 536
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 537
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 538
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 539
Phase: Bottleneck Computation, Dataset

Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 638
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 639
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 640
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 641
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 642
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 643
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 644
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 645
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 646
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 647
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 648
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 649
Phase: Bottleneck Computation, Dataset used: Validation, Image Index: 650
Phase: Bottleneck Computation, Dataset

Phase: Training, Dataset used: Validation, Batch Processed Count:  68, Epoch:  20, Accuracy:          1
Saver not created because there are no variables in the graph to restore
Restoring parameters from C:\Users\Admin\AppData\Local\Temp\05mxgu5u.w4t\custom_retrained_model_based_on_resnet_v2_50_299.meta
Froze 2 variables.
Converted 2 variables to const ops.


#### Evaluate the model

In [31]:
var predicitions = model.Transform(testSet);
var metrics = mlContext.MulticlassClassification.Evaluate(predicitions, labelColumnName: "LabelAsKey", predictedLabelColumnName: "PredictedLabel");

In [32]:
var confusionMatrix = Chart.Plot(new Graph.Heatmap 
    {
        x = categories.Distinct(),
        y = categories.Distinct().Reverse(),
        z = metrics.ConfusionMatrix.Counts.Reverse(),   
        zmin = 0,
        zmax = 5,
        colorscale = "off"
    });

confusionMatrix.WithLayout(new Layout.Layout { title="Confusion Matrix" });
confusionMatrix

#### Save the model

In [33]:
mlContext.Model.Save(model, trainSet.Schema, "imageClassifier.zip");